In [1]:
from sympy import *
import numpy as np
from pandas import *

In [2]:
x = Symbol("x")
a = 2
b = 4
c = 3
n = 51

In [3]:
f = ((a+53-n)*x**4 + (b - 53 + n) * x**2 + c) / ((x+1)*(x**2+1))
f

(4*x**4 + 2*x**2 + 3)/((x + 1)*(x**2 + 1))

In [4]:
k = 20
a = 0
b = 2
h = (b - a) / k

## Найдем решение интеграла

In [50]:
integrate(f)

2*x**2 - 4*x + 9*log(x + 1)/2 - 5*log(x**2 + 1)/4 + 5*atan(x)/2

In [51]:
integrate(f, (x, a, b))

-5*log(5)/4 + 5*atan(2)/2 + 9*log(3)/2

In [101]:
answer = round(integrate(f, (x, a, b)), 6)
answer

5.699830

##  Равномерная сетка

In [70]:
t_r = np.arange(0, 2.1, h)
t_r_df = DataFrame({"сетка":t})
t_r_df

,сетка
0,0.0
1,0.1
2,0.2
3,0.3
4,0.4
5,0.5
6,0.6
7,0.7
8,0.8
9,0.9


### Центрально равномерная сетка

In [72]:
t = np.array([(t_r[i] + t_r[i+1])/2 for i in range(k)])
t_df = DataFrame({"сетка":t})
t_df

,сетка
0,0.05
1,0.15
2,0.25
3,0.35
4,0.45
5,0.55
6,0.65
7,0.75
8,0.85
9,0.95


### Значения в узлах этой центрально равномерной сетки

In [78]:
y = [f.subs(x, t[i]) for i in range(k)]
y_df = DataFrame({"значения в узлах":y})
y_df

,значения в узлах
0,2.85479159244745
1,2.59128308706283
2,2.36470588235294
3,2.18099480326652
4,2.04689941931321
5,1.96694941489691
6,1.94238696277361
7,1.97142857142857
8,2.05013925391284
9,2.17339533003134


### Значения в узлах этой  равномерной сетки

In [92]:
y_r = [f.subs(x, t_r[i]) for i in range(k+1)]
y_r_df = DataFrame({"значения в узлах":y_r})
y_r_df

,значения в узлах
0,3.00000000000000
1,2.71863186318632
2,2.47307692307692
3,2.26704304869442
4,2.10738916256158
5,2.00000000000000
6,1.94779411764706
7,1.95041452822740
8,2.00487804878049
9,2.10654259959291


## Метод прямоугольников
$\int_a^b f(\tau)d\tau=h*(f(\Theta_1)+...+f(\Theta_k)+O(h)$

In [66]:
integral1 = np.array([h*y[i] for i in range(k)]).sum()
integral1

5.69716433698022

## Метод трапеций
$\int_a^b f(\tau)d\tau=h*(\frac{1}{2}*f(\Theta_0)+f(\Theta_1)+...+f(\Theta_{k-1})+\frac{1}{2}*f(\Theta_k))+O(h^2)$

In [80]:
integral2 = h * y_r[0] * 1/2
for i in range(1, k):
    integral2+=h*y_r[i]
integral2 += 1/2 * h * y_r[k]

In [81]:
integral2

5.70516154681846

### Квадратурные формулы парабол
$\int_a^b f(\tau)d\tau=\frac{р}{3}*(f(\Theta_0)+4*f(\Theta_1)+2*f(\Theta_2)+4*f(\Theta_2)+...+4f(\Theta_{k-1})+f(\Theta_k))+O(h^3)$

In [98]:
integral3 = y_r.copy()
for i in range(1, k):
    if i % 2 == 1:
        integral3[i] = integral3[i] * 4
    else:
        integral3[i] *= 2
        
integral3 = np.array(integral3).sum() * h/3

In [99]:
integral3

5.69983522329171

# Запишем полученные значения в таблицу

In [105]:
result = DataFrame({"аналитическое":answer, 
                    "метод прям.": integral1, 
                    "метод трапеции": integral2,
                    "квадр. формулы парабол": integral3}, 
                  index=range(1))

In [106]:
result

,аналитическое,метод прям.,метод трапеции,квадр. формулы парабол
0,5.699830,5.69716433698022,5.70516154681846,5.69983522329171


# 2 часть
Вычислим приближенно интеграл, пологая, что $\newline\int_a^b f(\tau)d\tau=\int_a^b L_k(\tau)d\tau$

### Для чебышевской сетки

Построим чебышевскую сетку по формуле
$B = <t_j=\frac{a+b}{2} - \frac{b-a}{2} * cos\frac{(2j+1)*\pi}{2(k+1)}:j=\overline{0,k}>$

In [13]:
koef1 = (a + b)/2
koef2 = (b - a)/2
B = []
for j in range(11):
    temp = koef1 - koef2 * np.cos(((2 * j  + 1) * np.pi)/22)
    B.append(temp)

In [14]:
fB = [f.subs(x, i) for i in B]
print("Значения в узлах сетки:\n")
for i in fB:
    print(i)

Значения в узлах сетки:

2.96966949704690
2.74417844111918
2.37664005071437
2.03707600320236
1.95661358208377
2.25000000000000
2.83261012186511
3.53312601095944
4.19316731591407
4.69571795067827
4.96542900834485


In [15]:
from numpy import linalg as LA
A = []
for i in B:
    temp = []
    for j in range(11):
        temp.append(i ** j)
    A.append(temp)
A = np.array(A)

In [16]:
A_inv = LA.inv(A)

In [17]:
x_ = np.dot(A_inv, fB)
x_df = DataFrame({"Коэффициенты полинома":x_})
x_df

,Коэффициенты полинома
0,3.00005356498424
1,-3.00649223943304
2,2.13088052491504
3,-3.02979773529921
4,11.0953937393965
5,-16.0311639277379
6,12.6552067399416
7,-6.06428674725612
8,1.77006833455107
9,-0.290421207521490


### Интеграл будет иметь вид 
$\int L_k(\tau) d\tau = x_0 \tau + x_1 \frac{\tau^2}{2} + ... + x_k \frac{\tau^{k+1}}{k+1} + C, где x_o, ..., x_k, k = 10 - коэффициенты полинома$ 
$\int_a^b L_k(\tau) d\tau = x_0 (2-0) + x_1 \frac{(2-0)^2}{2} + ... + x_{10} \frac{(2-0)^{11}}{11}$

In [18]:
L_integral = 0
for i in range(11):
    L_integral += x_[i]* 2**(i+1) / (i + 1)
L_integral

5.69982928705673